In [129]:
import pdfplumber
import pandas as pd
import os
import glob
from datetime import datetime
import re


In [130]:
with pdfplumber.open('DATA\\SOURCE_FILES\\data.pdf') as pdf:
    tables = []
    for page in pdf.pages:
        table = page.extract_table()
        if table:
            tables.extend(table)

# Convert tables to Pandas DataFrame
df = pd.DataFrame(tables)

  # To get place name
for index, row in df.iterrows():
    if 'Site Name' in row.values:
        # Get the index of 'Site Name' column
        site_name_index = row.index[row == 'Site Name'][0]

        # Get the place name from the next row in the same column
        place_name = df.iloc[index + 1, site_name_index].strip().replace(' ','_')

        break  # Exit loop after finding 'Site Name'
      
    # df['Date'] = df[2].str.extract(r'(\d{2}.\d{2}.\d{4})', expand=False)
    df['Date'] = datetime.strptime(re.search(r'(\d{2}\.\d{2}\.\d{4})', df.iloc[0, 2]).group(1), "%d.%m.%Y")


  #   # Remove date data from item name
    df[3] = df[3].str.replace(r'\n\d{2}\.\d{2}\.\d{4}\n\w{4}$', '', regex=True)

df=df[8:]
# df

In [131]:
# Find the index of the first occurrence of 'Sr.No'
first_occurrence_index = df[0].tolist().index('Sr.No')

# Filter out rows where the first column has 'Sr.No', except for the first occurrence
df_filtered = pd.concat([df.iloc[first_occurrence_index:first_occurrence_index+1], df[df[0] != 'Sr.No']])

# Reset the index of the DataFrame
df_filtered.reset_index(drop=True, inplace=True)

df=df_filtered

# Find the index of the first occurrence of 'Grand Total of Qty'
first_grand_total_index = df[df[0] == 'Grand Total of Qty'].index.tolist()

if first_grand_total_index:
    # If 'Grand Total of Qty' exists in the DataFrame
    first_grand_total_index = first_grand_total_index[0]
    # Slice the DataFrame to include rows only up to the first occurrence of 'Grand Total of Qty'
    df_filtered = df.iloc[:first_grand_total_index ]
else:
    # If 'Grand Total of Qty' does not exist in the DataFrame, keep the original DataFrame
    df_filtered = df.copy()
    
df=df_filtered


In [132]:
# df['Date'] = pd.to_datetime(df[3].str.extract(r'(\d{2}.\d{2}.\d{4})', expand=False),format="%d.%m.%Y")

df['StoreName'] = place_name

# Extract the date from the 'Date' column
date_from_column = df.at[1, 'Date']

# Remove date data from item name
df[3] = df[3].str.replace(r'\n\d{2}.\d{2}.\d{4}\n\w{4}$', '', regex=True)

# Remove columns 0, 6, 8 and 9
df = df.drop(columns=[0, 6, 8, 9])

#Remove the 1st row
df = df.iloc[1:]

# Add proper column names to the DataFrame
df.columns = ['HSN_CODE', 'ITEM_CODE', 'Item',  'Quantity', 'UOM','Price','Total', 'Date', 'StoreName']

# Data Cleaning
df['Total'] = df['Total'].str.replace(',','')
df['Item'] = df['Item'].str.strip("._/\n")
# df['ITEM_CODE'] = df['ITEM_CODE'].str.replace('\n','')

# Change data types of the DataFrame columns
df = df.astype({
    'Quantity': 'float',
    'Price': 'float',
    'Total': 'float'
})
df

,HSN_CODE,ITEM_CODE,Item,Quantity,UOM,Price,Total,Date,StoreName
1,590000001\n08081000,1982,APPLE FUJI (KG),19.6,KG,310.0,6076.00,2024-03-15,KEOTA_CC_RRL
2,590000002\n08081000,590000002,APPLE RED DELICIOUS (KG),17.3,KG,265.0,4584.50,2024-03-15,KEOTA_CC_RRL
3,590000005\n08081000,590000005,APPLE ROYAL GALA (KG),31.3,KG,270.0,8451.00,2024-03-15,KEOTA_CC_RRL
4,590000051\n08071100,590000051,MUSK MELON (KG),9.6,KG,85.0,816.00,2024-03-15,KEOTA_CC_RRL
5,590000068\n08071100,590000068,PAPAYA DISCO,142.0,KG,54.5,7739.00,2024-03-15,KEOTA_CC_RRL
6,590000083\n08109030,590000083,SAPOTA ROUND (KG),4.9,KG,90.0,441.00,2024-03-15,KEOTA_CC_RRL
7,590000092\n07020000,590000092,TOMATO,147.0,KG,35.0,5145.00,2024-03-15,KEOTA_CC_RRL
8,590000131\n07092000,590000131,GARLIC INDIAN (KG),49.9,KG,165.0,8233.50,2024-03-15,KEOTA_CC_RRL
9,590000147\n07092000,590000147,GREEN PEAS (KG),64.0,KG,45.0,2880.00,2024-03-15,KEOTA_CC_RRL
10,590000174\n07092000,590000174,BOTTLE GOURD (KG),135.5,KG,32.0,4336.00,2024-03-15,KEOTA_CC_RRL
